<a href="https://colab.research.google.com/github/elichen/aoc2022/blob/main/Day_19_Not_Enough_Minerals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
lines = '''Blueprint 1:  Each ore robot costs 4 ore.  Each clay robot costs 2 ore.  Each obsidian robot costs 3 ore and 14 clay.  Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2:  Each ore robot costs 2 ore.  Each clay robot costs 3 ore.  Each obsidian robot costs 3 ore and 8 clay.  Each geode robot costs 3 ore and 12 obsidian.'''
lines = lines.split('\n')

In [ ]:
lines = open("input.txt").readlines()

In [ ]:
lines = [x.strip() for x in lines]

In [ ]:
import re
import math
from tqdm import tqdm

In [ ]:
starting_state = {"or":1, "cr":0, "br":0, "gr":0,
                  "o":0, "c":0, "b":0, "g":0}
starting_t = 24
def update_state(m, s):
  s['o'] += s['or'] * m
  s['c'] += s['cr'] * m
  s['b'] += s['br'] * m
  s['g'] += s['gr'] * m
  return s
# time left, blueprint, state, history
import pdb
def simulate(t, b, s, h):
  if t < 1:
    return s['g']
  next_list = []
  # if next build ore robot
  if s['or'] <= max(([b['or']['o'], b['cr']['o'], b['br']['o'], b['gr']['o']])):
    # import pdb; pdb.set_trace()
    material_needed = max(b['or']['o'] - s['o'], 0)
    mins_to_wait = math.ceil(material_needed/s['or']) + 1
    if mins_to_wait < t:
      new_state = update_state(mins_to_wait, s.copy())
      new_state['o'] -= b['or']['o']
      new_state['or'] += 1
      next_list.append(simulate(t-mins_to_wait, b, new_state, h+[f"{t-mins_to_wait} or"]))
  # if next build clay robot
  if s['cr'] <= b['br']['c']:
    material_needed = max(b['cr']['o'] - s['o'], 0)
    mins_to_wait = math.ceil(material_needed/s['or']) + 1
    if mins_to_wait < t:
      new_state = update_state(mins_to_wait, s.copy())
      new_state['o'] -= b['cr']['o']
      new_state['cr'] += 1
      next_list.append(simulate(t-mins_to_wait, b, new_state, h+[f"{t-mins_to_wait} cr"]))
  # if next build obs robot
  if s['cr'] > 0:
    material_needed = max(b['br']['o'] - s['o'], 0)
    mins_to_wait = math.ceil(material_needed/s['or']) + 1
    material_needed_2 = b['br']['c'] - s['c']
    mins_to_wait_2 = math.ceil(material_needed_2/s['cr']) + 1
    mins_to_wait = mins_to_wait if mins_to_wait > mins_to_wait_2 else mins_to_wait_2
    if mins_to_wait < t:
      new_state = update_state(mins_to_wait, s.copy())
      new_state['o'] -= b['br']['o']
      new_state['c'] -= b['br']['c']
      new_state['br'] += 1
      next_list.append(simulate(t-mins_to_wait, b, new_state, h+[f"{t-mins_to_wait} br"]))
  # if next build geo robot
  if s['br'] > 0:
    material_needed = max(b['gr']['o'] - s['o'], 0)
    mins_to_wait = math.ceil(material_needed/s['or'])+1
    material_needed_2 = b['gr']['b'] - s['b']
    mins_to_wait_2 = math.ceil(material_needed_2/s['br'])+1
    mins_to_wait = mins_to_wait if mins_to_wait > mins_to_wait_2 else mins_to_wait_2
    if mins_to_wait < t:
      new_state = update_state(mins_to_wait, s.copy())
      new_state['o'] -= b['gr']['o']
      new_state['b'] -= b['gr']['b']
      new_state['gr'] += 1
      next_list.append(simulate(t-mins_to_wait, b, new_state, h+[f"{t-mins_to_wait} gr"]))
  if len(next_list) == 0:
    new_state = s.copy()
    update_state(t, new_state)
    next_list.append(simulate(0, b, new_state, h))
  return max(next_list)

In [ ]:
blueprint_format = {"or":{"o":0},
                    "cr":{"o":0},
                    "br":{"o":0,"c":0},
                    "gr":{"o":0,"b":0}}
quality = 0
for line in tqdm(lines):
  b = blueprint_format.copy()
  (id, b['or']['o'],
   b['cr']['o'],
   b['br']['o'], b['br']['c'],
   b['gr']['o'], b['gr']['b'])  = list(map(int,re.findall(r'\d+', line)))
  quality += id * simulate(24, b, starting_state, [])
quality

100%|██████████| 30/30 [00:34<00:00,  1.15s/it]


1115

In [ ]:
blueprint_format = {"or":{"o":0},
                    "cr":{"o":0},
                    "br":{"o":0,"c":0},
                    "gr":{"o":0,"b":0}}
prod = 1
for line in tqdm(lines[:3]):
  b = blueprint_format.copy()
  (id, b['or']['o'],
   b['cr']['o'],
   b['br']['o'], b['br']['c'],
   b['gr']['o'], b['gr']['b'])  = list(map(int,re.findall(r'\d+', line)))
  prod *= simulate(32, b, starting_state, [])
prod

 33%|███▎      | 1/3 [02:21<04:43, 141.85s/it]